In [1]:
import sys
sys.path.append("../mypkg")
from constants import RES_ROOT, FIG_ROOT, DATA_ROOT

In [2]:
%load_ext autoreload
%autoreload 2
# 0,1, 2, 3, be careful about the space

In [3]:

from argparse import ArgumentParser

import torch

import fastmri
from fastmri.data import transforms
from fastmri.models import VarNet

from fastmri.pl_modules import MriModule

In [4]:
import matplotlib.pyplot as plt
import time
from collections import defaultdict
from pathlib import Path

import numpy as np
import requests
import torch
from tqdm import tqdm

import fastmri
import fastmri.data.transforms as T
from fastmri.data import SliceDataset
from fastmri.models import VarNet
from fastmri.data.subsample import create_mask_for_mask_type
from my_pl_modules.varnet_module_Ysq import VarNetModuleYsq

In [5]:
model1_path = DATA_ROOT / 'pretrained_model' / 'brain_leaderboard_state_dict.pt'
model2_path = RES_ROOT / 'demo.ckpt';

In [6]:
device = "cuda";
model1 = VarNet(num_cascades=12, pools=4, chans=18, sens_pools=4, sens_chans=8);
model1.load_state_dict(torch.load(model1_path));
model1 = model1.eval();
model1 = model1.to(device);

model2_raw = VarNetModuleYsq.load_from_checkpoint(model2_path);
model2 = model2_raw.varnet
model2 = model2.eval();
model2 = model2.to(device);

In [7]:
params1 = model1.state_dict()
params2 = model2.state_dict()

for key in params1.keys():
    if key in params2:
        if torch.equal(params1[key], params2[key]):
            print(f"Parameter {key} is the same in both models.")
        else:
            print(f"Parameter {key} is different.")
    else:
        print(f"Parameter {key} not found in model2.")

differences = {}
for key in params1.keys():
    if key in params2:
        diff = torch.norm(params1[key] - params2[key])
        differences[key] = diff.item()
        print(f"Difference in {key}: {diff.item()}")

Parameter sens_net.norm_unet.unet.down_sample_layers.0.layers.0.weight is the same in both models.
Parameter sens_net.norm_unet.unet.down_sample_layers.0.layers.4.weight is the same in both models.
Parameter sens_net.norm_unet.unet.down_sample_layers.1.layers.0.weight is the same in both models.
Parameter sens_net.norm_unet.unet.down_sample_layers.1.layers.4.weight is the same in both models.
Parameter sens_net.norm_unet.unet.down_sample_layers.2.layers.0.weight is the same in both models.
Parameter sens_net.norm_unet.unet.down_sample_layers.2.layers.4.weight is the same in both models.
Parameter sens_net.norm_unet.unet.down_sample_layers.3.layers.0.weight is the same in both models.
Parameter sens_net.norm_unet.unet.down_sample_layers.3.layers.4.weight is the same in both models.
Parameter sens_net.norm_unet.unet.conv.layers.0.weight is the same in both models.
Parameter sens_net.norm_unet.unet.conv.layers.4.weight is the same in both models.
Parameter sens_net.norm_unet.unet.up_conv.